# Docker 

## download_miniconda.sh

In [4]:
# %load download_miniconda.sh
#!/bin/bash
# 检测系统架构
ARCH=$(uname -m)
MINICONDA_URL=""
if [ "$ARCH" = "x86_64" ]; then
    # 如果是x86_64架构
    MINICONDA_URL="https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh"
elif [ "$ARCH" = "aarch64" ]; then
    # 如果是ARM架构
    MINICONDA_URL="https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh"
else
    echo "不支持的架构: $ARCH"
    exit 1
fi
# 下载Miniconda安装脚本
wget $MINICONDA_URL -O miniconda.sh



## Dockerfile

In [3]:
# %load Dockerfile2
# 设置基础镜像
#         本地源地址更换 替换或更新 pip310_install  和pip_install 文件
#         机器变更 只需要调整  registerd.json host_config.json 的内容就行
#         账号密码管理 暂无
FROM ubuntu:latest
COPY download_miniconda.sh /download_miniconda.sh
SHELL ["/bin/bash", "-c"]
RUN apt-get update \
     && DEBIAN_FRONTEND=noninteractive apt-get install libglib2.0-0 -y \
	 && apt-get install -y build-essential wget vim gcc rsync sshpass git libgl1-mesa-glx 
RUN echo "下载并安装miniconda" \
	&& chmod +x /download_miniconda.sh \
	&& bash /download_miniconda.sh \
    && bash miniconda.sh -b -p /opt/conda \
    && rm -rf /download_miniconda.sh \
    && rm -rf miniconda.sh \
	&& ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh
RUN echo "启动conda 修改配置" \
	&& echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && source /root/.bashrc \
	&& source /opt/conda/bin/activate && echo "source /opt/conda/bin/activate" >> ~/.bashrc \
	&& conda create -n py310 python=3.10 -y \
	&& /opt/conda/envs/py310/bin/pip install ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/bin/pip install jupyterlab==3.6 ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/envs/py310/bin/python -m ipykernel install --user --name py310 --display-name py310
RUN echo "配置pip源" \
	&& /opt/conda/bin/pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple/ \
	&& /opt/conda/bin/pip config set global.trusted-host pypi.tuna.tsinghua.edu.cn
RUN echo "配置jupyter" \
    && /opt/conda/bin/jupyter lab --generate-config \
	&& echo "c.ServerApp.allow_origin = '*'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo "c.ServerApp.ip = '0.0.0.0'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo '{"IdentityProvider": {"hashed_password": "argon2:$argon2id$v=19$m=10240,t=10,p=8$I5lrjVBORbgeyAYwXUcAWA$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE"}}' > /root/.jupyter/jupyter_server_config.json \
	&& echo "c.ServerApp.password = 'argon2:\$argon2id\$v=19\$m=10240,t=10,p=8\$I5lrjVBORbgeyAYwXUcAWA\$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE'">> /root/.jupyter/jupyter_lab_config.py \
 	&& echo "c.ServerApp.notebook_dir = '/home'" >> /root/.jupyter/jupyter_lab_config.py \
CMD ["/opt/conda/bin/conda", "run", "jupyter", "lab", "--ip='0.0.0.0'", "--port=8888", "--no-browse", "--allow-root"]
RUN echo "docker 构建完毕 运行命令: docker run --gpus all -p<port>:8888 -v<work_path>:/home --name <name> -d jupyter:latest"



In [2]:
# %load Dockerfile
# 设置基础镜像
# 几个变动  magic_lib.py 可以更新 随即替换文件即可
#         本地源地址更换 替换或更新 pip310_install  和pip_install 文件
#         更新jupyter 启动项  ~/.ipython/profile_default/startup/ 去对应文件夹下查看
#         机器变更 只需要调整  registerd.json host_config.json 的内容就行
#         账号密码管理 暂无
FROM ubuntu:20.04
COPY packages /packages
COPY packages/pip_install.sh /usr/local/bin/pip_install
COPY packages/pip310_install.sh /usr/local/bin/pip310_install
COPY packages/pip310.sh /usr/local/bin/pip310
COPY packages/pipbase.sh /usr/local/bin/pipbase
COPY packages/magic_lib.py /root/.ipython/profile_default/startup/magic_lib.py
SHELL ["/bin/bash", "-c"]
RUN apt-get update \
    && DEBIAN_FRONTEND=noninteractive apt-get install libglib2.0-0 -y \
	&& apt-get install -y wget vim gcc rsync sshpass git libgl1-mesa-glx \
    && chmod +x /usr/local/bin/* /packages/*
RUN echo "下载并安装miniconda" \
	&& bash /packages/download_miniconda.sh \
    && bash miniconda.sh -b -p /opt/conda \
    && rm -rf /packages/download_miniconda.sh \
    && rm -rf miniconda.sh \
	&& ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh
RUN echo "启动conda 修改配置" \
	&& echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && source /root/.bashrc \
	&& source /opt/conda/bin/activate && echo "source /opt/conda/bin/activate" >> ~/.bashrc \
	&& conda create -n py310 python=3.10 -y \
	&& /opt/conda/envs/py310/bin/pip install ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/bin/pip install jupyterlab==3.6 ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/envs/py310/bin/python -m ipykernel install --user --name py310 --display-name py310
RUN echo "配置pip源" \
	&& /opt/conda/bin/pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple/ \
	&& /opt/conda/bin/pip config set global.trusted-host pypi.tuna.tsinghua.edu.cn
RUN echo "配置jupyter" \
    && /opt/conda/bin/jupyter lab --generate-config \
	&& echo "c.ServerApp.allow_origin = '*'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo "c.ServerApp.ip = '0.0.0.0'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo '{"IdentityProvider": {"hashed_password": "argon2:$argon2id$v=19$m=10240,t=10,p=8$I5lrjVBORbgeyAYwXUcAWA$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE"}}' > /root/.jupyter/jupyter_server_config.json \
	&& echo "c.ServerApp.password = 'argon2:\$argon2id\$v=19\$m=10240,t=10,p=8\$I5lrjVBORbgeyAYwXUcAWA\$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE'">> /root/.jupyter/jupyter_lab_config.py \
 	&& echo "c.ServerApp.notebook_dir = '/home'" >> /root/.jupyter/jupyter_lab_config.py \
 	&& echo "import os;os.chdir('/Users/zhaoxuefeng/GitHub/')" >> /root/.ipython/profile_default/startup/init.py # 添加jupyter启动项
CMD ["/opt/conda/bin/conda", "run", "jupyter", "lab", "--ip='0.0.0.0'", "--port=8888", "--no-browse", "--allow-root"]
RUN echo "docker 构建完毕 运行命令: docker run --gpus all -p<port>:8888 -v<work_path>:/home --name <name> -d jupyter:latest"

